<a href="https://colab.research.google.com/github/rjhalliday/python-llm/blob/main/langchain_wikipedia_llm_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simple RAG using wikipedia data retrieval

Retrieval Augmented Generation, or RAG, is an architectural approach that can improve the efficacy of large language model (LLM) applications by leveraging custom data. LLM's are trained on publically available data, however there is much data that is not publically available on the internet, such as commercial data.

A common usecase of an RAG would be where an organisation wants to leverage their internal proprietary documentation, databases and other information repositories to build an LLM which can answer questions specitic to their organisation. An example might be an aircraft manufacturer wanting to have an internal chatbot that employees or perhaps customers could query for information relating to their aircraft.

In this example I retrieve data from wikipedia to build a RAG. This is a more advanced version than my original [single document RAG you can find here](https://github.com/rjhalliday/python-llm/blob/main/langchain_simple_rag_with_gemini.ipynb).
The steps are
1.  Search wikipedia using the query "Artificial Intelligence". You can see a simple example of [Wikipedia retrieval in Python here](https://github.com/rjhalliday/python-examples/blob/main/python_wikipedia_api_data_retrieval.ipynb)
2. The results of this search is used to create a vector store.
3. The RAG is then queries for the question "What is Artificial Intelligence?"



In [1]:
# wikipedia has it's own unique markdown which we need to remove
!pip -q install mwparserfromhell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-google-genai mwparserfromhell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00


In [2]:
!pip install -qU langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [5]:
!pip install -qU chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 9.5 MB/s eta 0:00:00

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY



In [ ]:
import requests
import mwparserfromhell
from langchain.chains import RetrievalQA
#from langchain.document_loaders import Document
from langchain.schema import Document  # Import Document from langchain.schema

from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the language model using GoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Initialize embeddings
embeddings = GooglePalmEmbeddings()

def search_wikipedia(query):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={query}&format=json"
    try:
        response = requests.get(search_url)
        response.raise_for_status()  # Raise an error for HTTP issues
        data = response.json()
        page_titles = [result['title'] for result in data['query']['search']]
        return page_titles
    except requests.RequestException as e:
        print(f"Error during Wikipedia search: {e}")
        return []

def fetch_wikipedia_content(title):
    content_url = f"https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles={title}&rvprop=content&format=json"
    try:
        response = requests.get(content_url)
        response.raise_for_status()  # Raise an error for HTTP issues
        data = response.json()
        pages = data['query']['pages']
        page = next(iter(pages.values()))
        if 'revisions' in page:
            content = page['revisions'][0]['*']
            return content
        else:
            print(f"No content found for title: {title}")
            return ""
    except requests.RequestException as e:
        print(f"Error fetching content for {title}: {e}")
        return ""

def clean_wikipedia_content(content):
    wikicode = mwparserfromhell.parse(content)
    text = wikicode.strip_code()
    return text

def create_documents(query):
    titles = search_wikipedia(query)
    documents = []
    for title in titles:
        print(f"Fetching content for title: {title}")
        content = fetch_wikipedia_content(title)
        cleaned_content = clean_wikipedia_content(content)
        # Create Document objects from the cleaned content
        documents.append(Document(page_content=cleaned_content, metadata={"title": title}))
    return documents

# Create documents based on a query
query = "Artificial Intelligence"
documents = create_documents(query)

# Text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Creating a Vector Store
db = Chroma.from_documents(texts, embeddings)

# Retriever setup
retriever = db.as_retriever()

# QA Chain Setup
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# Execute a Query
query = "What is Artificial Intelligence?"
result = qa_chain({"query": query})
print(result["result"])


Fetching content for title: Artificial intelligence
Fetching content for title: Generative artificial intelligence
Fetching content for title: A.I. Artificial Intelligence
Fetching content for title: Artificial general intelligence
Fetching content for title: Applications of artificial intelligence
Fetching content for title: Ethics of artificial intelligence
Fetching content for title: History of artificial intelligence
Fetching content for title: Artificial intelligence in healthcare
Fetching content for title: Hallucination (artificial intelligence)
Fetching content for title: Timeline of artificial intelligence


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs. 

